In [1]:
import csv
import time
import os
import subprocess
import psutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


class CustomCallback(keras.callbacks.Callback):
    def __init__(self):
        super(CustomCallback, self).__init__()
        self.training_time = None
        self.average_memory_utilization = None
        self.average_gpu_utilization = None
        
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        self.memory_utilizations = []
        self.gpu_utilizations = []
        
    def on_train_end(self, logs={}):
        self.training_time = time.time() - self.start_time
        self.average_memory_utilization = np.mean(self.memory_utilizations)
        self.average_gpu_utilization = np.mean(self.gpu_utilizations)
        print('\nTotal training time: {} seconds'.format(self.training_time))
        print('\nAverage memory utilization during training so far: {}'.format(self.average_memory_utilization))
        print('\nAverage GPU utilization during training so far: {}'.format(self.average_gpu_utilization))

    def on_epoch_end(self, epoch, logs={}):
        memory_utilization = psutil.virtual_memory().used / pow(10, 9) # ex. 8.6Gi
        gpu_utilization_process = subprocess.Popen(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader'], stdout=subprocess.PIPE)
        gpu_utilization = int(gpu_utilization_process.stdout.readline().strip().decode('utf-8').split()[0])
        self.memory_utilizations.append(memory_utilization)
        self.gpu_utilizations.append(gpu_utilization)

        
def Classify(batch_size, epochs):
    # Load the CIFAR10 dataset
    (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

    # Preprocess the data
    x_train = x_train / 255.0
    x_test = x_test / 255.0

    # Build the model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    print("\n◆Train:")
    callback = CustomCallback()
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), callbacks=[callback])
    print("Done")
    
    # Evaluate the model
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print("Test Accuracy:", test_acc)
    
    with open('cifar10_classification.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow([batch_size, epochs, callback.average_memory_utilization, callback.average_gpu_utilization, callback.training_time, test_loss, test_acc])

        
for batch in range(100, 801, 100):
    for epoch in range(10, 41, 10):
        Classify(batch, epoch)



2023-02-10 06:41:41.896117: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-10 06:41:41.914967: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-10 06:41:41.915185: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-10 06:41:41.915817: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags


◆Train:
Epoch 1/10


2023-02-10 06:41:44.082647: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8400
2023-02-10 06:41:44.523376: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-10 06:41:44.523939: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-10 06:41:44.523960: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-02-10 06:41:44.524471: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-10 06:41:44.524529: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


72/72 [==============================] - 3s 26ms/step - loss: 2.0443 - accuracy: 0.2387 - val_loss: 1.7266 - val_accuracy: 0.3694
Epoch 2/10
72/72 [==============================] - 1s 20ms/step - loss: 1.7607 - accuracy: 0.3476 - val_loss: 1.5716 - val_accuracy: 0.4305
Epoch 3/10
72/72 [==============================] - 1s 20ms/step - loss: 1.6126 - accuracy: 0.4070 - val_loss: 1.4467 - val_accuracy: 0.4792
Epoch 4/10
72/72 [==============================] - 1s 20ms/step - loss: 1.5181 - accuracy: 0.4463 - val_loss: 1.3678 - val_accuracy: 0.5130
Epoch 5/10
72/72 [==============================] - 1s 21ms/step - loss: 1.4512 - accuracy: 0.4753 - val_loss: 1.3147 - val_accuracy: 0.5292
Epoch 6/10
72/72 [==============================] - 1s 20ms/step - loss: 1.4135 - accuracy: 0.4926 - val_loss: 1.2759 - val_accuracy: 0.5546
Epoch 7/10
72/72 [==============================] - 1s 20ms/step - loss: 1.3642 - accuracy: 0.5126 - val_loss: 1.2453 - val_accuracy: 0.5572
Epoch 8/10
72/72 [======